In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.ticker as ticker
import rpy2.robjects as robjects


In [2]:
robjects.r('devtools::install_github("josedv82/airball")')
robjects.r('library(airball)')
result = robjects.r(nba_travel())

R[write to console]: Downloading GitHub repo josedv82/airball@HEAD



These packages have more recent versions available.
It is recommended to update all of them.
Which would you like to update?

1: All                       
2: CRAN packages only        
3: None                      
4: sp (1.6-0 -> 1.6-1) [CRAN]

Enter one or more numbers, or an empty line to skip updates: 1
sp (1.6-0 -> 1.6-1) [CRAN]


R[write to console]: Skipping 1 packages not available: nbastatR

R[write to console]: Installing 2 packages: sp, nbastatR

R[write to console]: trying URL 'https://cloud.r-project.org/bin/macosx/big-sur-arm64/contrib/4.3/sp_1.6-1.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2098680 bytes (2.0 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 


The downloaded binary packages are in
	/var/folders/3k/6lf4ln2n63b7tvgn2yck__yc0000gn/T//RtmpD2qO9f/downloaded_packages
── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/private/var/folders/3k/6lf4ln2n63b7tvgn2yck__yc0000gn/T/RtmpD2qO9f/remotes144eb26259a30/josedv82-airball-a62db42/DESCRIPTION’ ... OK
* preparing ‘airball’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘airball_0.4.4.1.tar.gz’



ERROR: dependency ‘nbastatR’ is not available for package ‘airball’
* removing ‘/Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/library/airball’
R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: package ‘nbastatR’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages 

R[write to console]: 2: 
R[write to console]: In i.p(...) :
R[write to console]: 
 
R[write to console]:  installation of package ‘/var/folders/3k/6lf4ln2n63b7tvgn2yck__yc0000gn/T//RtmpD2qO9f/file144eb577992e4/airball_0.4.4.1.tar.gz’ had non-zero exit status

R[write to console]: Error in library(airball) : there is no package called ‘airball’



RRuntimeError: Error in library(airball) : there is no package called ‘airball’


In [ ]:
injuryData = pd.read_csv("~/Documents/Job Search - 2023/NBA_Injury_Predictor/Injuries.csv")

In [ ]:
injuryData.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
injuryData.rename(columns={'Notes': 'Injury Type'}, inplace=True)
injuryData.rename(columns={'Relinquished': 'Player'}, inplace=True)

### We want to separate the dataset by year to get a good idea of the total number of injuries that have occured annually

In [ ]:
# Setting the objects in the column to a pandas datetime object
injuryData['Date'] = pd.to_datetime(injuryData['Date'])

#Grouping rows by 'decade' column
injuryData['Year'] = (injuryData['Date'].dt.year)



### Let's now remove all rows where the players have been acquired back from the IL 


In [ ]:
# Removing the Acquired column removes all duplicates 
injuryData = injuryData[injuryData['Acquired'].isna()]
del injuryData['Acquired']

### Now let's calculate the total sum of injuries for each Year

## `It is important to note that the NBA did not have and IR/IL until the 1989-1990 season. Prior to this injured players were still members of the active roster. For this reason, it's likely many injuries do not appear in the dataset.`

In [ ]:
# We only want to include data from the years after 1989 
yearly_injuries = {}
total_injuries = 0
for year, group in injuryData.groupby('Year'):
    if year > 1989: 
        injuries = group.shape[0]
        yearly_injuries[year] = injuries
        total_injuries += injuries
        
df = pd.DataFrame(yearly_injuries.items(), columns=['Year', 'Injuries'])
condition1 = df['Year'].astype(int) <= 2015
df.loc[condition1, 'Players'] = 354
df['isleagueExpansion'] = np.where(df['Year']<=2015, 0, 1)

# Set the condition for the second range of years (1996 onwards)
condition2 = df['Year'].astype(int) > 2015
df.loc[condition2, 'Players'] = 450
    

In [ ]:
# adding the total number of games played in the NBA each year to the Dataframe
def get_games_played(df):

    games_played = {
      year: 6620
      for year in range(1990, 2024)
  }

  # Update the dictionary for years with a different number of games played.
    games_played[1999] = 4100
    games_played[2011] = 4096
    games_played[2020] = 4384
    games_played[2021] = 4384

  # Create a new column in the DataFrame for the number of games played.
    df['Games Played'] = df.Year.map(games_played)
    df['Games Played'] = df['Games Played'].astype(int)

get_games_played(df)

In [ ]:
# Adding feature to account for a lockout season
df['Lockout'] = np.where(df['Year'].isin([1999, 2011]), 1, 0)

In [ ]:
# Add feature for average number of miles travelled year over year
annual_travel = pd.read_csv("~/Documents/Job Search - 2023/NBA_Injury_Predictor/AverageTravel.csv")
df = pd.concat([df, annual_travel['Miles Traveled']], axis=1)

In [ ]:
# why are there so few injuries in the early in the early 90s
# use a proxy to get data
# average NBA salary 


In [ ]:
def normalize_players(df):
  # Get the mean and standard deviation of the players feature.
    mu = df['Players'].mean()
    std = df['Players'].std()

  # Normalize the players feature.
    df['Normalized Players'] = (df['Players'] - mu) / std
    
    return df

normalize_players(df)


In [ ]:
# Attempting to use R package to get data for total number of miles travelled
reticulate.import_from_github("josedv82/airball")

In [ ]:
# calculating the correlation for the two features I will use to train my model
Players_corr = df['Players'].corr(df['Injuries'])
Normalized_corr = df['Normalized Players'].corr(df['Injuries'])
Year_corr = df['Year'].corr(df['Injuries'])


print("Correlation for Players feature:", Players_corr)
print("Correlation for Year feature:", Year_corr)
print("Correlation for Players feature:", Normalized_corr)


In [ ]:
#Using Linear Regression function to fit our model
train_data = df[:26]

X_train = np.array(train_data[['Year', 'Normalized Players']]).reshape(-1, 2)
y_train = train_data['Injuries']
reg = LinearRegression()


fit_model = reg.fit(X_train, y_train)
print(df.shape)
X_train.shape


In [ ]:
# using a scikit-learn attribute, the coef represents the wieghts assigned to different features 
print(f"The Weight of feature(s): {fit_model.coef_}")

In [ ]:
# getting the intercept(bias) for our fitted model
print(f"The Bias of the model: {fit_model.intercept_}")

In [ ]:
# Adding our insample prediction for each year to our training data set
train_data.loc[:, 'prediction_insample'] = fit_model.predict(X_train).copy()
train_data.head()

In [ ]:
# Calculating the error, MSE, and ABS error for each year
train_data['error'] = train_data.prediction_insample - train_data.Injuries
train_data['squared_error'] = train_data['error']**2
train_data['abs'] = abs(train_data['error'])

train_data

In [ ]:
# I'm not sure why my error is zero 

# Returns the mean for error, squared error and absolute error for the training set
train_data[['error', 'squared_error', 'abs']].mean()

In [ ]:
sns.set(style="darkgrid")
sns.lineplot(x=train_data['Year'], y=train_data['Injuries'], data=train_data)
sns.lineplot(x=train_data['Year'], y=train_data['prediction_insample'], data=train_data)
plt.xlabel('Year')
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries Year by Year')


### <u>Time to test on a new set of data:<u>

In [ ]:
test_data = df[26:]
 
X_test = np.array(test_data[['Year', 'Players']]).reshape([-1, 2])
y_test = test_data['Injuries']
X_test

In [ ]:
y_pred = fit_model.predict(X_test)
y_pred
# Adding our insample prediction for each year to our test data set
test_data.loc[:, 'prediction_insample'] = fit_model.predict(X_test).copy()
test_data.head()

In [ ]:
test_data['error'] = test_data.prediction_insample - test_data.Injuries
test_data['squared_error'] = test_data['error']**2
test_data['abs'] = abs(test_data['error'])

test_data

In [ ]:
df.describe()["Injuries"]

In [ ]:

plt.plot(X_test[:, 0], y_pred, 'r-', label='Predicted Data')
sns.set(style="darkgrid")
sns.lineplot(x=test_data['Year'], y=test_data['Injuries'], data=test_data)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries Year by Year')
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.show()

In [ ]:
# Now let's remove all the rows from the data frame that have data before 1990
injuryData = injuryData[injuryData['Year'] >= 1990]